# Data Preparation

Importing the libraries and the dataset.

In [1]:
import os
import pandas as pd
import numpy as np

notebook = 'Data_preparation.ipnyb'
path = os.path.dirname(os.path.abspath(notebook))+'/data/'
train_path = path+'census_income_learn.csv'
test_path = path+'census_income_test.csv'

#columns names for mapping according to the metadata file
column_names =['age',
               'class of worker',
               'detailed industry recode',
               'detailed occupation recode',
               'education',
               'wage per hour',
               'enrolled in edu inst last wk',
               'marital stat',
               'major industry code',
               'major occupation code',
               'race',
               'hispanic origin',
               'sex',
               'member of a labor union',
               'reason for unemployment',
               'full or part time employment stat',
               'capital gains',
               'capital losses',
               'dividends from stocks',
               'tax filer status',
               'region of previous residence',
               'state of previous residence',
               'detailed household and family stat',
               'detailed household summary in household',
               'instance weight',
               'migration code-change in msa',
               'migration code-change in reg',
               'migration code-move within reg',
               'live in this house 1 year ago',
               'migration prev res in sunbelt',
               'num persons worked for employer',
               'family members under 18',
               'country of birth father',
               'country of birth mother',
               'country of birth self',
               'citizenship',
               'own business or self employed',
               'fill inc questionnaire for veterans admin',
               'veterans benefits',
               'weeks worked in year',
              'year',
              'target']


train = pd.read_csv(train_path,names=column_names)
test = pd.read_csv(test_path,names=column_names)

In this notebook we will summarize the steps performed in the Exploratory Data Analysis notebook.

For details on the why they are being made please take a look at the EDA notebook.

In [2]:
#method to be used in both the train and test preparation
def create_dummy_df(df, cat_cols, dummy_na):
    '''
    INPUT:
    df - pandas dataframe with categorical variables you want to dummy
    cat_cols - list of strings that are associated with names of the categorical columns
    dummy_na - Bool holding whether you want to dummy NA vals of categorical columns or not
    
    OUTPUT:
    df - a new dataframe that has the following characteristics:
            1. contains all columns that were not specified as categorical
            2. removes all the original columns in cat_cols
            3. dummy columns for each of the categorical columns in cat_cols
            4. if dummy_na is True - it also contains dummy columns for the NaN values
            5. Use a prefix of the column name with an underscore (_) for separating 
    '''
    for col in  cat_cols:
        try:
            #for each cat add dummy var, drop original column
            df = pd.concat([df.drop(columns=col, axis=1), pd.get_dummies(df[col], prefix=col, prefix_sep='_', drop_first=True, dummy_na=dummy_na)], axis=1)
        except:
            continue
    return df;

## Data cleaning pipeline

In [3]:
#removing features
columns_removing = ['instance weight','migration code-change in msa','migration code-change in reg',
                    'migration code-move within reg','migration prev res in sunbelt',
                   'detailed industry recode','detailed occupation recode',
                    'fill inc questionnaire for veterans admin', 'detailed household and family stat',
                    'state of previous residence','family members under 18','year']

train = train.loc[:,~train.columns.isin(columns_removing)]

#filtering age 18+
train = train.loc[train.loc[:,'age']>=18]


#duplicate entries
index = np.setdiff1d(train.index, train.drop_duplicates().index)

duplicate_index = train.loc[index].loc[(train.loc[index].loc[:,'dividends from stocks'] > 0)
                    | (train.loc[index].loc[:,'capital gains'] > 0)
                    | (train.loc[index].loc[:,'capital losses'] > 0)
                    | ((train.loc[index].loc[:,'weeks worked in year'] > 0)
                    & (train.loc[index].loc[:,'weeks worked in year'] < 52))].index.to_list()

train = train.loc[~train.index.isin(duplicate_index)]


#handling missing values
missing_values_index = train.loc[(train.loc[:,'country of birth father'] ==' ?')
                  | (train.loc[:,'country of birth mother'] ==' ?')
                  | (train.loc[:,'country of birth self'] ==' ?')
                  | (train.loc[:,'country of birth self'] ==' ?')].index.to_list()


train = train.loc[~train.index.isin(missing_values_index)]
#train.isin([' ?']).sum().sum()

#feature engineering
#transforming the target column
map_target = {' 50000+.':1,' - 50000.':0}
train.target = train.target.map(map_target)

#transforming continuos variables into categories
bins = [0, 0.000001,np.inf]
labels = [0,1]

#categorization
train['capital gains'] = pd.cut(train['capital gains'], bins,labels=labels, include_lowest=True)
train['capital losses']= pd.cut(train['capital losses'], bins,labels=labels, include_lowest=True)
train['dividends from stocks']= pd.cut(train['dividends from stocks'], bins,labels=labels, include_lowest=True)
train['wage per hour'] = pd.cut(train['wage per hour'], bins,labels=labels, include_lowest=True)

#changing variable types to integers
train['capital gains'] = train['capital gains'].cat.codes
train['capital losses'] = train['capital losses'].cat.codes
train['dividends from stocks'] = train['dividends from stocks'].cat.codes
train['wage per hour'] = train['wage per hour'].cat.codes

#same process with different bins and labels
bins = [0, 0.000001,51.99999999, 52]
labels = [0,1,2]

train['weeks worked in year'] = pd.cut(train['weeks worked in year'], bins,labels=labels, include_lowest=True)
train['weeks worked in year'] = train['weeks worked in year'].cat.codes


#recategorization of the country and citizenship features
list_of_keys = train['country of birth father'].unique()
list_of_values = np.zeros(len(train['country of birth father'].unique()))
map_country_father = dict(zip(list_of_keys, list_of_values))
map_country_father[' United-States'] = 1
train['country of birth father'] = train['country of birth father'].map(map_country_father)
train['country of birth father'] = train['country of birth father'].astype(int)


list_of_keys = train['country of birth mother'].unique()
list_of_values = np.zeros(len(train['country of birth mother'].unique()))
map_country_mother = dict(zip(list_of_keys, list_of_values))
map_country_mother[' United-States'] = 1
train['country of birth mother'] = train['country of birth mother'].map(map_country_mother)
train['country of birth mother'] = train['country of birth mother'].astype(int)


list_of_keys = train['country of birth self'].unique()
list_of_values = np.zeros(len(train['country of birth self'].unique()))
map_country_self = dict(zip(list_of_keys, list_of_values))
map_country_self[' United-States'] = 1
train['country of birth self'] = train['country of birth self'].map(map_country_self)
train['country of birth self'] = train['country of birth self'].astype(int)


map_citenzship ={' Native- Born in the United States':1,
       ' Foreign born- Not a citizen of U S ':0,
       ' Foreign born- U S citizen by naturalization':0,
       ' Native- Born abroad of American Parent(s)':1,
       ' Native- Born in Puerto Rico or U S Outlying':1}
train['citizenship'] = train['citizenship'].map(map_citenzship)
train['citizenship'] = train['citizenship'].astype(int)


#one hot encoding the remaining category type variables
dummy_cols = ['class of worker', 'education','enrolled in edu inst last wk', 
              'marital stat', 'major industry code',
       'major occupation code', 'race', 'hispanic origin', 'sex',
       'member of a labor union', 'reason for unemployment',
       'full or part time employment stat',
              'tax filer status',
       'region of previous residence',
       'detailed household summary in household',
            'live in this house 1 year ago', 'num persons worked for employer',
             'own business or self employed', 'veterans benefits'
       ]

train = create_dummy_df(train,dummy_cols, False) 

In [4]:
pd.set_option("max_columns", train.shape[1])
train.head()

,age,wage per hour,capital gains,capital losses,dividends from stocks,country of birth father,country of birth mother,country of birth self,citizenship,weeks worked in year,target,class of worker_ Local government,class of worker_ Never worked,class of worker_ Not in universe,class of worker_ Private,class of worker_ Self-employed-incorporated,class of worker_ Self-employed-not incorporated,class of worker_ State government,class of worker_ Without pay,education_ 11th grade,education_ 12th grade no diploma,education_ 1st 2nd 3rd or 4th grade,education_ 5th or 6th grade,education_ 7th and 8th grade,education_ 9th grade,education_ Associates degree-academic program,education_ Associates degree-occup /vocational,education_ Bachelors degree(BA AB BS),education_ Doctorate degree(PhD EdD),education_ High school graduate,education_ Less than 1st grade,education_ Masters degree(MA MS MEng MEd MSW MBA),education_ Prof school degree (MD DDS DVM LLB JD),education_ Some college but no degree,enrolled in edu inst last wk_ High school,enrolled in edu inst last wk_ Not in universe,marital stat_ Married-A F spouse present,marital stat_ Married-civilian spouse present,marital stat_ Married-spouse absent,marital stat_ Never married,marital stat_ Separated,marital stat_ Widowed,major industry code_ Armed Forces,major industry code_ Business and repair services,major industry code_ Communications,major industry code_ Construction,major industry code_ Education,major industry code_ Entertainment,major industry code_ Finance insurance and real estate,major industry code_ Forestry and fisheries,major industry code_ Hospital services,major industry code_ Manufacturing-durable goods,major industry code_ Manufacturing-nondurable goods,major industry code_ Medical except hospital,major industry code_ Mining,major industry code_ Not in universe or children,major industry code_ Other professional services,major industry code_ Personal services except private HH,major industry code_ Private household services,major industry code_ Public administration,major industry code_ Retail trade,major industry code_ Social services,major industry code_ Transportation,major industry code_ Utilities and sanitary services,major industry code_ Wholesale trade,major occupation code_ Armed Forces,major occupation code_ Executive admin and managerial,major occupation code_ Farming forestry and fishing,major occupation code_ Handlers equip cleaners etc,major occupation code_ Machine operators assmblrs & inspctrs,major occupation code_ Not in universe,major occupation code_ Other service,major occupation code_ Precision production craft & repair,major occupation code_ Private household services,major occupation code_ Professional specialty,major occupation code_ Protective services,major occupation code_ Sales,major occupation code_ Technicians and related support,major occupation code_ Transportation and material moving,race_ Asian or Pacific Islander,race_ Black,race_ Other,race_ White,hispanic origin_ Central or South American,hispanic origin_ Chicano,hispanic origin_ Cuban,hispanic origin_ Do not know,hispanic origin_ Mexican (Mexicano),hispanic origin_ Mexican-American,hispanic origin_ NA,hispanic origin_ Other Spanish,hispanic origin_ Puerto Rican,sex_ Male,member of a labor union_ Not in universe,member of a labor union_ Yes,reason for unemployment_ Job loser - on layoff,reason for unemployment_ New entrant,reason for unemployment_ Not in universe,reason for unemployment_ Other job loser,reason for unemployment_ Re-entrant,full or part time employment stat_ Full-time schedules,full or part time employment stat_ Not in labor force,full or part time employment stat_ PT for econ reasons usually FT,full or part time employment stat_ PT for econ reasons usually PT,full or part time employment stat_ PT for non-econ reasons usually FT,full or part time employment stat_ Unemployed full-time,full or part time employment stat_ Unemployed part- time,tax filer status_ Joint both 6

Before applying the same data processing to the test set let's standardize the age column.

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

standarization_columns = ['age']

transformed_data = train.loc[:,standarization_columns].copy()

ct = ColumnTransformer([
    ('Standarization', StandardScaler(), standarization_columns)
], remainder='passthrough')


transformed_data.loc[:,standarization_columns] = ct.fit_transform(transformed_data)
train[standarization_columns] = transformed_data


train.head()

,age,wage per hour,capital gains,capital losses,dividends from stocks,country of birth father,country of birth mother,country of birth self,citizenship,weeks worked in year,target,class of worker_ Local government,class of worker_ Never worked,class of worker_ Not in universe,class of worker_ Private,class of worker_ Self-employed-incorporated,class of worker_ Self-employed-not incorporated,class of worker_ State government,class of worker_ Without pay,education_ 11th grade,education_ 12th grade no diploma,education_ 1st 2nd 3rd or 4th grade,education_ 5th or 6th grade,education_ 7th and 8th grade,education_ 9th grade,education_ Associates degree-academic program,education_ Associates degree-occup /vocational,education_ Bachelors degree(BA AB BS),education_ Doctorate degree(PhD EdD),education_ High school graduate,education_ Less than 1st grade,education_ Masters degree(MA MS MEng MEd MSW MBA),education_ Prof school degree (MD DDS DVM LLB JD),education_ Some college but no degree,enrolled in edu inst last wk_ High school,enrolled in edu inst last wk_ Not in universe,marital stat_ Married-A F spouse present,marital stat_ Married-civilian spouse present,marital stat_ Married-spouse absent,marital stat_ Never married,marital stat_ Separated,marital stat_ Widowed,major industry code_ Armed Forces,major industry code_ Business and repair services,major industry code_ Communications,major industry code_ Construction,major industry code_ Education,major industry code_ Entertainment,major industry code_ Finance insurance and real estate,major industry code_ Forestry and fisheries,major industry code_ Hospital services,major industry code_ Manufacturing-durable goods,major industry code_ Manufacturing-nondurable goods,major industry code_ Medical except hospital,major industry code_ Mining,major industry code_ Not in universe or children,major industry code_ Other professional services,major industry code_ Personal services except private HH,major industry code_ Private household services,major industry code_ Public administration,major industry code_ Retail trade,major industry code_ Social services,major industry code_ Transportation,major industry code_ Utilities and sanitary services,major industry code_ Wholesale trade,major occupation code_ Armed Forces,major occupation code_ Executive admin and managerial,major occupation code_ Farming forestry and fishing,major occupation code_ Handlers equip cleaners etc,major occupation code_ Machine operators assmblrs & inspctrs,major occupation code_ Not in universe,major occupation code_ Other service,major occupation code_ Precision production craft & repair,major occupation code_ Private household services,major occupation code_ Professional specialty,major occupation code_ Protective services,major occupation code_ Sales,major occupation code_ Technicians and related support,major occupation code_ Transportation and material moving,race_ Asian or Pacific Islander,race_ Black,race_ Other,race_ White,hispanic origin_ Central or South American,hispanic origin_ Chicano,hispanic origin_ Cuban,hispanic origin_ Do not know,hispanic origin_ Mexican (Mexicano),hispanic origin_ Mexican-American,hispanic origin_ NA,hispanic origin_ Other Spanish,hispanic origin_ Puerto Rican,sex_ Male,member of a labor union_ Not in universe,member of a labor union_ Yes,reason for unemployment_ Job loser - on layoff,reason for unemployment_ New entrant,reason for unemployment_ Not in universe,reason for unemployment_ Other job loser,reason for unemployment_ Re-entrant,full or part time employment stat_ Full-time schedules,full or part time employment stat_ Not in labor force,full or part time employment stat_ PT for econ reasons usually FT,full or part time employment stat_ PT for econ reasons usually PT,full or part time employment stat_ PT for non-econ reasons usually FT,full or part time employment stat_ Unemployed full-time,full or part time employment stat_ Unemployed part- time,tax filer status_ Joint both 6

The following step:

test.isin([' ?']).sum().sum()

To check if there are more missing values in columns different than in the train dataset was performed where indicated in the comments.

The result was 0, so nothing further needs to be done.

In [7]:
#removing features
columns_removing = ['instance weight','migration code-change in msa','migration code-change in reg',
                    'migration code-move within reg','migration prev res in sunbelt',
                   'detailed industry recode','detailed occupation recode',
                    'fill inc questionnaire for veterans admin', 'detailed household and family stat',
                    'state of previous residence','family members under 18','year']

test = test.loc[:,~test.columns.isin(columns_removing)]

#filtering age 18+
test = test.loc[test.loc[:,'age']>=18]


#duplicate entries
index = np.setdiff1d(test.index, test.drop_duplicates().index)

duplicate_index = test.loc[index].loc[(test.loc[index].loc[:,'dividends from stocks'] > 0)
                    | (test.loc[index].loc[:,'capital gains'] > 0)
                    | (test.loc[index].loc[:,'capital losses'] > 0)
                    | ((test.loc[index].loc[:,'weeks worked in year'] > 0)
                    & (test.loc[index].loc[:,'weeks worked in year'] < 52))].index.to_list()

test = test.loc[~test.index.isin(duplicate_index)]


#handling missing values
missing_values_index = test.loc[(test.loc[:,'country of birth father'] ==' ?')
                  | (test.loc[:,'country of birth mother'] ==' ?')
                  | (test.loc[:,'country of birth self'] ==' ?')
                  | (test.loc[:,'country of birth self'] ==' ?')].index.to_list()


test = test.loc[~test.index.isin(missing_values_index)]
#this is where the step to check further missing values was performed

#feature engineering
#transforming the target column
map_target = {' 50000+.':1,' - 50000.':0}
test.target = test.target.map(map_target)

#transforming continuos variables into categories
bins = [0, 0.000001,np.inf]
labels = [0,1]

#categorization
test['capital gains'] = pd.cut(test['capital gains'], bins,labels=labels, include_lowest=True)
test['capital losses']= pd.cut(test['capital losses'], bins,labels=labels, include_lowest=True)
test['dividends from stocks']= pd.cut(test['dividends from stocks'], bins,labels=labels, include_lowest=True)
test['wage per hour'] = pd.cut(test['wage per hour'], bins,labels=labels, include_lowest=True)

#changing variable types to integers
test['capital gains'] = test['capital gains'].cat.codes
test['capital losses'] = test['capital losses'].cat.codes
test['dividends from stocks'] = test['dividends from stocks'].cat.codes
test['wage per hour'] = test['wage per hour'].cat.codes

#same process with different bins and labels
bins = [0, 0.000001,51.99999999, 52]
labels = [0,1,2]

test['weeks worked in year'] = pd.cut(test['weeks worked in year'], bins,labels=labels, include_lowest=True)
test['weeks worked in year'] = test['weeks worked in year'].cat.codes


#recategorization of the country and citizenship features
list_of_keys = test['country of birth father'].unique()
list_of_values = np.zeros(len(test['country of birth father'].unique()))
map_country_father = dict(zip(list_of_keys, list_of_values))
map_country_father[' United-States'] = 1
test['country of birth father'] = test['country of birth father'].map(map_country_father)
test['country of birth father'] = test['country of birth father'].astype(int)


list_of_keys = test['country of birth mother'].unique()
list_of_values = np.zeros(len(test['country of birth mother'].unique()))
map_country_mother = dict(zip(list_of_keys, list_of_values))
map_country_mother[' United-States'] = 1
test['country of birth mother'] = test['country of birth mother'].map(map_country_mother)
test['country of birth mother'] = test['country of birth mother'].astype(int)


list_of_keys = test['country of birth self'].unique()
list_of_values = np.zeros(len(test['country of birth self'].unique()))
map_country_self = dict(zip(list_of_keys, list_of_values))
map_country_self[' United-States'] = 1
test['country of birth self'] = test['country of birth self'].map(map_country_self)
test['country of birth self'] = test['country of birth self'].astype(int)


map_citenzship ={' Native- Born in the United States':1,
       ' Foreign born- Not a citizen of U S ':0,
       ' Foreign born- U S citizen by naturalization':0,
       ' Native- Born abroad of American Parent(s)':1,
       ' Native- Born in Puerto Rico or U S Outlying':1}
test['citizenship'] = test['citizenship'].map(map_citenzship)
test['citizenship'] = test['citizenship'].astype(int)


#one hot encoding the remaining category type variables
dummy_cols = ['class of worker', 'education','enrolled in edu inst last wk', 
              'marital stat', 'major industry code',
       'major occupation code', 'race', 'hispanic origin', 'sex',
       'member of a labor union', 'reason for unemployment',
       'full or part time employment stat',
              'tax filer status',
       'region of previous residence',
       'detailed household summary in household',
            'live in this house 1 year ago', 'num persons worked for employer',
             'own business or self employed', 'veterans benefits'
       ]

test = create_dummy_df(test,dummy_cols, False) 

#using the scaler created above to transform the age column
test[standarization_columns] = ct.transform(test[standarization_columns])

In [8]:
test.head()

,age,wage per hour,capital gains,capital losses,dividends from stocks,country of birth father,country of birth mother,country of birth self,citizenship,weeks worked in year,target,class of worker_ Local government,class of worker_ Never worked,class of worker_ Not in universe,class of worker_ Private,class of worker_ Self-employed-incorporated,class of worker_ Self-employed-not incorporated,class of worker_ State government,class of worker_ Without pay,education_ 11th grade,education_ 12th grade no diploma,education_ 1st 2nd 3rd or 4th grade,education_ 5th or 6th grade,education_ 7th and 8th grade,education_ 9th grade,education_ Associates degree-academic program,education_ Associates degree-occup /vocational,education_ Bachelors degree(BA AB BS),education_ Doctorate degree(PhD EdD),education_ High school graduate,education_ Less than 1st grade,education_ Masters degree(MA MS MEng MEd MSW MBA),education_ Prof school degree (MD DDS DVM LLB JD),education_ Some college but no degree,enrolled in edu inst last wk_ High school,enrolled in edu inst last wk_ Not in universe,marital stat_ Married-A F spouse present,marital stat_ Married-civilian spouse present,marital stat_ Married-spouse absent,marital stat_ Never married,marital stat_ Separated,marital stat_ Widowed,major industry code_ Armed Forces,major industry code_ Business and repair services,major industry code_ Communications,major industry code_ Construction,major industry code_ Education,major industry code_ Entertainment,major industry code_ Finance insurance and real estate,major industry code_ Forestry and fisheries,major industry code_ Hospital services,major industry code_ Manufacturing-durable goods,major industry code_ Manufacturing-nondurable goods,major industry code_ Medical except hospital,major industry code_ Mining,major industry code_ Not in universe or children,major industry code_ Other professional services,major industry code_ Personal services except private HH,major industry code_ Private household services,major industry code_ Public administration,major industry code_ Retail trade,major industry code_ Social services,major industry code_ Transportation,major industry code_ Utilities and sanitary services,major industry code_ Wholesale trade,major occupation code_ Armed Forces,major occupation code_ Executive admin and managerial,major occupation code_ Farming forestry and fishing,major occupation code_ Handlers equip cleaners etc,major occupation code_ Machine operators assmblrs & inspctrs,major occupation code_ Not in universe,major occupation code_ Other service,major occupation code_ Precision production craft & repair,major occupation code_ Private household services,major occupation code_ Professional specialty,major occupation code_ Protective services,major occupation code_ Sales,major occupation code_ Technicians and related support,major occupation code_ Transportation and material moving,race_ Asian or Pacific Islander,race_ Black,race_ Other,race_ White,hispanic origin_ Central or South American,hispanic origin_ Chicano,hispanic origin_ Cuban,hispanic origin_ Do not know,hispanic origin_ Mexican (Mexicano),hispanic origin_ Mexican-American,hispanic origin_ NA,hispanic origin_ Other Spanish,hispanic origin_ Puerto Rican,sex_ Male,member of a labor union_ Not in universe,member of a labor union_ Yes,reason for unemployment_ Job loser - on layoff,reason for unemployment_ New entrant,reason for unemployment_ Not in universe,reason for unemployment_ Other job loser,reason for unemployment_ Re-entrant,full or part time employment stat_ Full-time schedules,full or part time employment stat_ Not in labor force,full or part time employment stat_ PT for econ reasons usually FT,full or part time employment stat_ PT for econ reasons usually PT,full or part time employment stat_ PT for non-econ reasons usually FT,full or part time employment stat_ Unemployed full-time,full or part time employment stat_ Unemployed part- time,tax filer status_ Joint both 6

In [9]:
train.shape, test.shape

((136669, 133), (68591, 133))

Both the train and test datasets have the same number of columns, but let's check if the list is the same and in the same order.

In [10]:
train.columns.tolist() == test.columns.tolist()

True

Since the condition is true we will store these processed datasets into new files in order to use them in the modelling notebook.

In [11]:
train.to_csv('data/train.csv',index=False)
test.to_csv('data/test.csv',index=False)

Plese move on to the modelling notebook for the next step.